In [2]:
import pickle as pkl
from copy import deepcopy

import pickle as pkl
import spacy
import csv
import json
from copy import deepcopy
import spacy_stanza
import benepar
from tqdm import tqdm
from copy import deepcopy

import stanza

In [3]:
def merge_maximum_span(spans):
    temp = deepcopy(spans)
    temp.sort(key=lambda x: x[1])
    to_pop = []
    for j, (word_0, start_idx_0, end_idx_0) in enumerate(temp):
        for k, (word_1, start_idx_1, end_idx_1) in enumerate(temp):
            if k == j:
                continue
            if (start_idx_1 >= start_idx_0) and (end_idx_1 <= end_idx_0):
                to_pop.append(temp[k])
    for item in to_pop:
        if item in temp:
            temp.remove(item)

    temp.sort(key=lambda x: x[1])

    return temp

In [4]:
with open('parsed_corpus_tbbt.pkl', 'rb') as f:
    data = pkl.load(f)

In [62]:
count = 0
sent_count = 0
for epi_id in data:
    if epi_id!=(1,1):
        continue
    episode = data[epi_id]
    for scene in episode:
        sent_count += len(scene)
        for item in scene:
            if item['en_subtitles']!="":
                utterance = item['en_subtitles']
            else:
                utterance = item['utterance']
            all_nps = sorted(list(set(item['sm_noun_chunk']) | set(item['berkeley_noun_chunk']) | set(item['trf_noun_chunk'])))
            merged_nps = merge_maximum_span(all_nps)
            if len(merged_nps)==len(all_nps):
                continue
            print(utterance)
            print()
            print(item['berkeley_pron'])
            # print("Noun Chunk:(SM, Berkeley, Roberta)")
            # print("Spacy SM :", item['sm_noun_chunk'])
            # print("Berkeley :", item['berkeley_noun_chunk'])
            # print("Roberta  :", item['trf_noun_chunk'])
            # print("All Nps :", all_nps)
            # print("Merged  :", merged_nps)
            # print("Removed :", set(all_nps)-set(merged_nps))
            print()
            print("=="*50)
            count += 1

One across is Aegean, eight down is Nabokov. Twenty-six across is MCM. Fourteen down is... Move your finger. phylum, which makes 14 across Port-Au-Prince. See, Papa Doc's capital idea, that's Port-Au-Prince. Haiti.

[('One', 0, 1, 'NUM', 'CD'), ('across', 1, 2, 'ADP', 'RB'), ('is', 2, 3, 'AUX', 'VBZ'), ('Aegean', 3, 4, 'PROPN', 'NNP'), (',', 4, 5, 'PUNCT', ','), ('eight', 5, 6, 'NUM', 'CD'), ('down', 6, 7, 'ADV', 'RB'), ('is', 7, 8, 'AUX', 'VBZ'), ('Nabokov', 8, 9, 'PROPN', 'NNP'), ('.', 9, 10, 'PUNCT', '.'), ('Twenty', 10, 11, 'NUM', 'CD'), ('-', 11, 12, 'PUNCT', 'HYPH'), ('six', 12, 13, 'NUM', 'CD'), ('across', 13, 14, 'ADP', 'IN'), ('is', 14, 15, 'AUX', 'VBZ'), ('MCM', 15, 16, 'PROPN', 'NNP'), ('.', 16, 17, 'PUNCT', '.'), ('Fourteen', 17, 18, 'NUM', 'CD'), ('down', 18, 19, 'ADV', 'RB'), ('is', 19, 20, 'AUX', 'VBZ'), ('...', 20, 21, 'PUNCT', ':'), ('Move', 21, 22, 'VERB', 'VB'), ('your', 22, 23, 'PRON', 'PRP$'), ('finger', 23, 24, 'NOUN', 'NN'), ('.', 24, 25, 'PUNCT', '.'), ('phylum'

In [10]:
sm_parser = spacy.load('en_core_web_sm')
berkeley_parser = spacy.load('en_core_web_md')
berkeley_parser.add_pipe("benepar", config={"model": "benepar_en3"})
trf_parser = spacy.load("en_core_web_trf")

In [14]:
stanza.download("en")
stanza_parser = spacy_stanza.load_pipeline("en")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2022-05-18 15:11:34 INFO: Downloading default packages for language: en (English)...
2022-05-18 15:11:36 INFO: File exists: /Users/boyuanzheng/stanza_resources/en/default.zip.
2022-05-18 15:11:40 INFO: Finished downloading models and saved to /Users/boyuanzheng/stanza_resources.
2022-05-18 15:11:40 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-05-18 15:11:40 INFO: Use device: cpu
2022-05-18 15:11:40 INFO: Loading: tokenize
2022-05-18 15:11:40 INFO: Loading: pos
2022-05-18 15:11:40 INFO: Loading: lemma
2022-05-18 15:11:40 INFO: Loading: depparse
2022-05-18 15:11:40 INFO: Loading: sentiment
2022-05-18 15:11:41 INFO: Loading: constituency
2022-05-18 15:11:41 INFO: Loading: ner
2022-05-18 15:11:42 INFO: Done loading pr

In [56]:
instance = {}
# text = "It is 5:30am in the morning."
text = "Hi. Anyway we brought home Indian food and I know that moving can be stressful and I find that when I'm undergoing stress that good food and company can have a comforting effect. Also, curry's a natural laxative and I don't have to tell you that, you know a clean colon is just one less thing to worry about."

instance['utterance'] = text
utterance = sm_parser(text)
instance['sm_noun_chunk'] = [(item.text, item.start, item.end) for item in utterance.noun_chunks]
instance['sm_pron'] = [(item.text, i, i + 1, item.pos_, item.tag_) for i, item in enumerate(utterance)]

utterance = berkeley_parser(text)
instance['berkeley_noun_chunk'] = [(item.text, item.start, item.end) for item in utterance.noun_chunks]
instance['berkeley_pron'] = [(item.text, i, i + 1, item.pos_, item.tag_) for i, item in enumerate(utterance)]

utterance = trf_parser(text)
instance['trf_noun_chunk'] = [(item.text, item.start, item.end) for item in utterance.noun_chunks]
instance['trf_pron'] = [(item.text, i, i + 1, item.pos_, item.tag_) for i, item in enumerate(utterance)]

/Users/boyuanzheng/.conda/envs/multi_coref/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
/Users/boyuanzheng/.conda/envs/multi_coref/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
# WDT, Noun Chunks
# Compare with the pilot
# What,

# Fix Constituency Parsing Result

In [27]:
count = 0
sent_count = 0
for epi_id in data:
    if epi_id!=(1,1):
        continue
    episode = data[epi_id]
    for scene in episode:
        sent_count += len(scene)
        for item in scene:
            if item['en_subtitles']!="":
                utterance = item['en_subtitles']
            else:
                utterance = item['utterance']
            all_nps = sorted(list(set(item['sm_noun_chunk']) | set(item['berkeley_noun_chunk']) | set(item['trf_noun_chunk'])))
            merged_nps = merge_maximum_span(all_nps)
            # if len(merged_nps)==len(all_nps):
            #     continue
            print(utterance)
            print()
            # print(item['berkeley_pron'])
            # print("Noun Chunk:(SM, Berkeley, Roberta)")
            # print("Spacy SM :", item['sm_noun_chunk'])
            # print("Berkeley :", item['berkeley_noun_chunk'])
            # print()

            # Collect Constituency Parsing Result
            doc = berkeley_parser(utterance)
            temp = []
            for sent in list(doc.sents):
                for token in sent._.constituents:
                    if len(token._.labels)==0:
                        continue
                    if token._.labels[0]!="NP":
                        continue
                    temp.append((token.text, token.start, token.end))
                    # print(token, token._.labels[0], token.start, token.end)
            print(temp)
            print(merge_maximum_span(temp))
            print(item['berkeley_noun_chunk'])
            print(len(merge_maximum_span(temp)), len(item['berkeley_noun_chunk']))

            # doc = stanza_parser(utterance)
            # for sent in list(doc.sents):
            #     for item in sent._.constituents:
            #         if len(item._.labels)==0:
            #             continue
            #         if item._.labels[0]!="NP":
            #             continue
            #         print(item, item._.labels[0], item.start, item.end)


            # print("Roberta  :", item['trf_noun_chunk'])
            # print("All Nps :", all_nps)
            # print("Merged  :", merged_nps)
            # print("Removed :", set(all_nps)-set(merged_nps))
            # print()
            print("=="*50)
            count += 1

If a photon is directed through a plane with two slits in it and either is observed it will not go through both. If unobserved, it will. If it's observed after it left the plane, before it hits its target... it will not have gone through both slits.



/Users/boyuanzheng/.conda/envs/multi_coref/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[('a photon', 1, 3), ('a plane with two slits in it', 6, 13), ('a plane', 6, 8), ('two slits in it', 9, 13), ('two slits', 9, 11), ('it', 12, 13), ('either', 14, 15), ('it', 17, 18), ('both', 22, 23), ('it', 27, 28), ('it', 31, 32), ('it', 35, 36), ('the plane', 37, 39), ('it', 41, 42), ('its target', 43, 45), ('it', 46, 47), ('both slits', 52, 54)]
[('a photon', 1, 3), ('a plane with two slits in it', 6, 13), ('either', 14, 15), ('it', 17, 18), ('both', 22, 23), ('it', 27, 28), ('it', 31, 32), ('it', 35, 36), ('the plane', 37, 39), ('it', 41, 42), ('its target', 43, 45), ('it', 46, 47), ('both slits', 52, 54)]
[('a photon', 1, 3), ('a plane', 6, 8), ('two slits', 9, 11), ('it', 12, 13), ('it', 17, 18), ('both', 22, 23), ('it', 27, 28), ('it', 31, 32), ('it', 35, 36), ('the plane', 37, 39), ('it', 41, 42), ('its target', 43, 45), ('it', 46, 47), ('both slits', 52, 54)]
13 14
Agreed. What's your point?

[('your point', 4, 6)]
[('your point', 4, 6)]
[('What', 2, 3), ('your point', 4, 6)]

In [7]:
with open('parsed_corpus_tbbt_constituent_berkeley_large.pkl', 'rb') as f:
    data = pkl.load(f)

In [8]:
for scene in data[(1,1)]:
    for utt in scene:
        for x in utt:
            print(x, ":", utt[x])
    print()

speaker : Sheldon
utterance : So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.
utterance_with_info : So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.
en_subtitles : If a photon is directed through a plane with two slits in it and either is observed it will not go through both. If unobserved, it will. If it's observed after it left the plane, before it hits its target... it will not have gone through both slits.
zh_subtitles : 将光子正对平面上的双缝 观察任意一个隙缝 它不会穿过那两个隙缝 如果没被观察 那就会 总之 如果观察它在离开平面到击中目标之前 它就不会穿过那两个隙缝
fa_subtitles : اگه يه فوتون به ط

In [60]:
utterance = berkeley_parser(text)

for sent in list(utterance.sents):
    print(sent)
    print(sent._.parse_string)
    print()

Hi.
(INTJ (UH Hi) (. .))

Anyway we brought home Indian food
(S (ADVP (RB Anyway)) (NP (PRP we)) (VP (VBD brought) (ADVP (RB home)) (NP (JJ Indian) (NN food))))

and I know that moving can be stressful and I find that when I'm undergoing stress that good food and company can have a comforting effect.
(S (CC and) (S (NP (PRP I)) (VP (VBP know) (SBAR (IN that) (S (NP (VBG moving)) (VP (MD can) (VP (VB be) (ADJP (JJ stressful)))))))) (CC and) (S (NP (PRP I)) (VP (VBP find) (SBAR (IN that) (S (SBAR (WHADVP (WRB when)) (S (NP (PRP I)) (VP (VBP 'm) (VP (VBG undergoing) (NP (NN stress)))))) (SBAR (NP (IN that) (NP (JJ good) (NN food) (CC and) (NN company))) (VP (MD can) (VP (VB have) (NP (DT a) (VBG comforting) (NN effect))))))))) (. .))

Also, curry's a natural laxative
(S (ADVP (RB Also)) (, ,) (NP (NN curry)) (VP (VBZ 's) (NP (DT a) (JJ natural) (NN laxative))))

and I don't have to tell you that, you know a clean colon is just one less thing to worry about.
(S (CC and) (S (NP (PRP I)) (VP

/Users/boyuanzheng/.conda/envs/multi_coref/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [58]:
for item in instance:
    print(item, ":", instance[item])

utterance : Hi. Anyway we brought home Indian food and I know that moving can be stressful and I find that when I'm undergoing stress that good food and company can have a comforting effect. Also, curry's a natural laxative and I don't have to tell you that, you know a clean colon is just one less thing to worry about.
sm_noun_chunk : [('we', 3, 4), ('Indian food', 6, 8), ('I', 9, 10), ('I', 17, 18), ('I', 21, 22), ('stress', 24, 25), ('good food', 26, 28), ('company', 29, 30), ('a comforting effect', 32, 35), ("Also, curry's a natural laxative", 36, 43), ('I', 44, 45), ('you', 50, 51), ('you', 53, 54), ('a clean colon', 55, 58), ('just one less thing', 59, 63)]
sm_pron : [('Hi', 0, 1, 'INTJ', 'UH'), ('.', 1, 2, 'PUNCT', '.'), ('Anyway', 2, 3, 'INTJ', 'UH'), ('we', 3, 4, 'PRON', 'PRP'), ('brought', 4, 5, 'VERB', 'VBD'), ('home', 5, 6, 'ADV', 'RB'), ('Indian', 6, 7, 'ADJ', 'JJ'), ('food', 7, 8, 'NOUN', 'NN'), ('and', 8, 9, 'CCONJ', 'CC'), ('I', 9, 10, 'PRON', 'PRP'), ('know', 10, 11, 'V

In [47]:
print(count, sent_count, count/sent_count)

1490 18746 0.07948362317294357


In [34]:
for item in data[(1,1)][0]:
    all_nps = sorted(list(set(item['sm_noun_chunk']) | set(item['berkeley_noun_chunk']) | set(item['trf_noun_chunk'])))
    merged_nps = merge_maximum_span(all_nps)
    if len(merged_nps)==len(all_nps):
        continue
    print(item['utterance'])
    print()
    print("Noun Chunk:(SM, Berkeley, Roberta)")
    print("Spacy SM :", item['sm_noun_chunk'])
    print("Berkeley :", item['berkeley_noun_chunk'])
    print("Roberta  :", item['trf_noun_chunk'])
    print("All Nps :", all_nps)
    print("Merged  :", merged_nps)
    print("Removed :", set(all_nps)-set(merged_nps))
    print()
    # print("Pron:(SM, Berkeley, Roberta)")
    # print([token for token in item['sm_pron'] if token[3]=="PRON"])
    # print([token for token in item['berkeley_pron'] if token[3]=="PRON"])
    # print([token for token in item['trf_pron'] if token[3]=="PRON"])
    # print("Merged Noun Phrases:")
    # print()
    # print("Merged Pronuns:")
    # print()
    print("=="*50)

One across is Aegean, eight down is Nabakov, twenty-six across is MCM, fourteen down is… move your finger… phylum, which makes fourteen across Port-au-Prince. See, Papa Doc’s capital idea, that’s Port-au-Prince. Haiti.

Noun Chunk:(SM, Berkeley, Roberta)
Spacy SM : [('Aegean', 3, 4), ('Nabokov', 8, 9), ('MCM', 15, 16), ('your finger', 22, 24), ('phylum', 25, 26), ('which', 27, 28), ('Port-Au-Prince', 31, 36), ("Papa Doc's capital idea", 39, 44), ('that', 45, 46), ('Port-Au-Prince', 47, 52), ('Haiti', 53, 54)]
Berkeley : [('Aegean', 3, 4), ('Nabokov', 8, 9), ('MCM', 15, 16), ('your finger', 22, 24), ('phylum', 25, 26), ('which', 27, 28), ('Port-Au', 31, 34), ('-', 34, 35), ('Prince', 35, 36), ("Papa Doc's capital idea", 39, 44), ('that', 45, 46), ('Port-Au-Prince', 47, 52), ('Haiti', 53, 54)]
Roberta  : [('Aegean', 3, 4), ('Nabokov', 8, 9), ('MCM', 15, 16), ('your finger', 22, 24), ('phylum', 25, 26), ('which', 27, 28), ('Port-Au-Prince', 31, 36), ('that', 45, 46), ('Port-Au-Prince', 47

In [ ]:
with open('tbbt_three_way.pkl', 'rb') as f:
    data = pkl.load(f)

In [3]:
with open('tbbt_three_way.pkl', 'rb') as f:
    data = pkl.load(f)

In [4]:
for x in data[(1,1)][2]:
    for item in x:
        print(item, ":", x[item])
    print()

speaker : Leonard
utterance : Okay, well, make yourself at home.
utterance_with_info : Okay, well, make yourself at home.
en_subtitles : Okay, well, make yourself at home.
zh_subtitles : 请随意
fa_subtitles : خب ،تو خونه راحت باش ممنونم

speaker : Penny
utterance : Okay, thankyou.
utterance_with_info : Okay, thankyou.
en_subtitles : 
zh_subtitles : 
fa_subtitles : 

speaker : Leonard
utterance : You’re very welcome.
utterance_with_info : You’re very welcome.
en_subtitles : Thank you. You're very welcome.
zh_subtitles : 
fa_subtitles : خواهش ميکنم

speaker : Penny
utterance : This looks like some serious stuff, Leonard, did you do this?
utterance_with_info : This looks like some serious stuff, Leonard, did you do this?
en_subtitles : This looks like some serious stuff. Leonard, did you do this?
zh_subtitles : 这些看起来好深奥啊 这是你写的吗?
fa_subtitles : او اين يه سري معادلات جدي به نظر مياد لئونارد اين ها رو تو نوشتي؟

speaker : Sheldon
utterance : Actually that’s my work.
utterance_with_info : Actual

In [18]:
with open('try.pkl', 'rb') as f:
    data = pkl.load(f)

In [19]:
for item in data:
    for scene in data[item]:
        for utt in scene:
            if utt['en_subtitles']!="":
                continue
            for x in utt:
                print(x, ":", utt[x])
            print()

speaker : Receptionist
utterance : Hang on.
utterance_with_info : Hang on.
en_subtitles : 
zh_subtitles : 
fa_subtitles : 
sm_noun_chunk : []
sm_pron : [('Hang', 0, 1, 'VERB', 'VB'), ('on', 1, 2, 'ADP', 'RP'), ('.', 2, 3, 'PUNCT', '.')]
berkeley_noun_chunk : []
berkeley_pron : [('Hang', 0, 1, 'VERB', 'VB'), ('on', 1, 2, 'ADP', 'RP'), ('.', 2, 3, 'PUNCT', '.')]
trf_noun_chunk : []
trf_pron : [('Hang', 0, 1, 'VERB', 'VB'), ('on', 1, 2, 'ADP', 'RP'), ('.', 2, 3, 'PUNCT', '.')]

speaker : Leonard
utterance : Okay.
utterance_with_info : Okay.
en_subtitles : 
zh_subtitles : 
fa_subtitles : 
sm_noun_chunk : []
sm_pron : [('Okay', 0, 1, 'INTJ', 'UH'), ('.', 1, 2, 'PUNCT', '.')]
berkeley_noun_chunk : []
berkeley_pron : [('Okay', 0, 1, 'INTJ', 'UH'), ('.', 1, 2, 'PUNCT', '.')]
trf_noun_chunk : []
trf_pron : [('Okay', 0, 1, 'INTJ', 'UH'), ('.', 1, 2, 'PUNCT', '.')]

speaker : Sheldon
utterance : Bye-bye
utterance_with_info : Bye-bye
en_subtitles : 
zh_subtitles : 
fa_subtitles : 
sm_noun_chunk : 

In [8]:
for scene in data[(1,1)]:
    for utt in scene:
        if utt['en_subtitles']!="":
            continue
        print(utt)
    print()

{'speaker': 'Phoebe Buffay', 'utterance': 'No.', 'utterance_with_info': 'No.', 'en_subtitles': '', 'zh_subtitles': '', 'fa_subtitles': '', 'sm_noun_chunk': [], 'sm_pron': [], 'berkeley_noun_chunk': [], 'berkeley_pron': [], 'trf_noun_chunk': [], 'trf_pron': []}
{'speaker': 'Ross Geller', 'utterance': 'Hi.', 'utterance_with_info': 'Hi.', 'en_subtitles': '', 'zh_subtitles': '', 'fa_subtitles': '', 'sm_noun_chunk': [], 'sm_pron': [], 'berkeley_noun_chunk': [], 'berkeley_pron': [], 'trf_noun_chunk': [], 'trf_pron': []}
{'speaker': 'Joey Tribbiani', 'utterance': 'Ohh.', 'utterance_with_info': 'Ohh.', 'en_subtitles': '', 'zh_subtitles': '', 'fa_subtitles': '', 'sm_noun_chunk': [], 'sm_pron': [], 'berkeley_noun_chunk': [], 'berkeley_pron': [], 'trf_noun_chunk': [], 'trf_pron': []}
{'speaker': 'Phoebe Buffay', 'utterance': 'Ooh! Oh!', 'utterance_with_info': 'Ooh! Oh!', 'en_subtitles': '', 'zh_subtitles': '', 'fa_subtitles': '', 'sm_noun_chunk': [], 'sm_pron': [], 'berkeley_noun_chunk': [], 'ber

In [6]:
for x in data[(1,1)][0]:
    for item in x:
        print(item, ":", x[item])
    print()

speaker : Monica Geller
utterance : There's nothing to tell! He's just some guy I work with!
utterance_with_info : There's nothing to tell! He's just some guy I work with!
en_subtitles : There's nothing to tell. It's just some guy I work with.
zh_subtitles : 没什么好说的！ 他不过是我的同事！
fa_subtitles : ! چيزي براي گفتن وجود نداره ! اون فقط يکي از همکارامه
sm_noun_chunk : [('nothing', 2, 3), ('It', 6, 7), ('some guy', 9, 11), ('I', 11, 12)]
sm_pron : [('There', 0, 1, 'PRON', 'EX'), ("'s", 1, 2, 'VERB', 'VBZ'), ('nothing', 2, 3, 'PRON', 'NN'), ('to', 3, 4, 'PART', 'TO'), ('tell', 4, 5, 'VERB', 'VB'), ('.', 5, 6, 'PUNCT', '.'), ('It', 6, 7, 'PRON', 'PRP'), ("'s", 7, 8, 'AUX', 'VBZ'), ('just', 8, 9, 'ADV', 'RB'), ('some', 9, 10, 'DET', 'DT'), ('guy', 10, 11, 'NOUN', 'NN'), ('I', 11, 12, 'PRON', 'PRP'), ('work', 12, 13, 'VERB', 'VBP'), ('with', 13, 14, 'ADP', 'IN'), ('.', 14, 15, 'PUNCT', '.')]
berkeley_noun_chunk : [('nothing', 2, 3), ('It', 6, 7), ('some guy', 9, 11), ('I', 11, 12)]
berkeley_pron : [